In [ ]:
import os
import ee
import geemap
import geedim
import geetools
import fiona
from osgeo import gdal
import pandas as pd
import geopandas as gpd 
import folium
from folium.plugins import MiniMap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

In [ ]:
#df = pd.DataFrame()

In [ ]:
cpes_path = 'C:/Users/00097030/LocalData/Spatial Data/CPES/geojson/geojsoncpes.geojson'

In [ ]:
cpes = gpd.read_file('C:/Users/00097030/LocalData/Spatial Data/CPES/shp/CPES_ag.shp')
cpes.to_file(cpes_path, driver='GeoJSON')

In [ ]:
ee.Initialize()

In [ ]:
geemap.show_youtube('OlNlqfj4uHo')

In [ ]:
####AOI######
path_to_aoi = 'C:/Users/00097030/LocalData/Spatial Data/Australia States/Australia_simple/WA_simple.shp'
AOI = geemap.shp_to_ee(path_to_aoi)
 

In [ ]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.add_minimap()
#minimap = MiniMap(tile_layer="Stamen Toner")
#minimap.add_to(Map)
Map

In [ ]:
with open(cpes_path) as f:
    json_data = json.load(f)
    
#GeoJSON() 

json_layer = GeoJSON(
    data=json_data,
    name='CPES',
    style = {'color': 'red',
             'weight': 2,
             'fillColor': 'red', 
             'fillOpacity': 0}
    #hover_style={'fillColor': 'red', 'fillOpacity': 0.5},
)



In [ ]:
ee_cpes = geojson_to_ee(json_data)


In [ ]:
Map.add_layer(ee_cpes)

In [ ]:
Map.draw_features

In [ ]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
#Map.addLayer(AOI, {}, 'ROI')
#Map


In [ ]:
# Set the date range
start_date = '2021-01-01'
end_date = '2022-01-31'

In [ ]:
# Create a Sentinel-2 image composite
image = geemap.dynamic_world_s2(AOI, start_date, end_date)
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.addLayer(image, vis_params, 'Sentinel-2 image')

In [ ]:
# Create Dynamic World land cover composite
# return_type='hillshade'
landcover = geemap.dynamic_world(AOI, start_date, end_date, return_type="class")

Map.addLayer(landcover, {}, 'Land Cover')
Map

In [ ]:
props = landcover.toDictionary().getInfo()
print(props)

bandNames = landcover.bandNames()
print('Band names: ', bandNames.getInfo())

In [ ]:
dwCol = (
    ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
    .filterBounds(AOI)
    .filterDate(start_date, end_date)
)


In [ ]:
CLASS_NAMES = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']

In [ ]:
VIS_PALETTE = [
    '419BDF', '397D49', '88B053', '7A87C6',
    'E49635', 'DFC35A', 'C4281B', 'A59B8F',
    'B39FE1']

In [ ]:
dwImage = ee.Image(dwCol.first())

#Map.addLayer(dwCol, {}, 'DW')
#Map

In [ ]:
top1Prob = dwImage.select(CLASS_NAMES).reduce(ee.Reducer.max())

In [ ]:
vis_params = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF",
        "#397D49",
        "#88B053",
        "#7A87C6",
        "#E49635",
        "#DFC35A",
        "#C4281B",
        "#A59B8F",
        "#B39FE1",
    ],
}

dwRgb = (
     dwImage
    .select('label')
    .visualize({vis_params})
    .divide(255)
)

In [ ]:
Map.addLayer(
    top1Prob,
    {min: 0, max: 0.65},
    'Dynamic World')

Map

In [ ]:
tn = (landcover.filter(ee.Filter.eq("NAME", 'Tennessee'))
    .first()
)


In [ ]:
# Add legend to the map
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
#Map

In [ ]:
fishnet = geemap.fishnet(AOI, h_interval=0.1, v_interval=0.1, delta=1)

In [ ]:
Map.addLayer(fishnet, {}, 'Fishnet 1')
Map

In [ ]:
out_dir = os.path.expanduser('C:/Users/00097030/LocalData/Spatial Data/Dynamic world')

In [ ]:
geemap.download_ee_image_tiles(
    landcover, fishnet, out_dir, prefix="LandCover_", crs="EPSG:4326", scale=20
)